In [75]:
import tweepy as tw
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from textblob import TextBlob
import re #regular #remove all the useless words in analytics

In [76]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## import Movie Data

In [69]:
def switch_mon(argument):
    switcher = {
        "Jan":'01',
        "Feb":'02',
        "Mar":'03',
        "Apr":'04',
        "May":'05',
        "Jun":'06',
        "Jul":'07',
        "Aug":'08',
        "Sep":'09',
        "Oct":'10',
        "Nov":'11',
        "Dec":'12'
    }
    return(switcher.get(argument, "Invalid month"))

In [658]:
revenue5 = pd.read_csv('1-177-boxoffice.csv',encoding = "ISO-8859-1")
movie=pd.read_csv('IMDBMovieData2.csv',encoding = "ISO-8859-1")

In [677]:
movie=pd.read_csv('IMDBMovieData2.csv',encoding = "ISO-8859-1")

In [659]:
revenue5.head()

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number
0,Guardians of the Galaxy,1-Aug,134390839,-,32939,134390839,1
1,Guardians of the Galaxy,8-Aug,63154982,-0.53,15449,197545821,2
2,Guardians of the Galaxy,15-Aug,36708036,-0.42,9929,234253857,3
3,Guardians of the Galaxy,22-Aug,24043411,-0.35,7132,258297268,4
4,Guardians of the Galaxy,29-Aug,26110586,0.09,7542,284407854,5


## transfer data format

In [660]:
revenue5['Percentage_Change']=pd.to_numeric(revenue5['Percentage_Change'], errors='coerce')
revenue5['Avg']=pd.to_numeric(revenue5['Avg'], errors='coerce')

In [661]:
revenue5.head()

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number
0,Guardians of the Galaxy,1-Aug,134390839,NaN,32939.0,134390839,1
1,Guardians of the Galaxy,8-Aug,63154982,-0.53,15449.0,197545821,2
2,Guardians of the Galaxy,15-Aug,36708036,-0.42,9929.0,234253857,3
3,Guardians of the Galaxy,22-Aug,24043411,-0.35,7132.0,258297268,4
4,Guardians of the Galaxy,29-Aug,26110586,0.09,7542.0,284407854,5


In [662]:
revenue5[revenue5['Avg'].isnull()]

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number
55,Split,5-May,149780,0.19,NaN,138291365,16
76,Sing,5-May,66380,0.73,NaN,270395425,20
218,Hidden Figures,3-Nov,220283,NaN,NaN,169607287,46
664,Lowriders,23-Jun,123605,0.01,NaN,6303560,7


In [656]:
revenue5

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number
55,Split,5-May,149780,0.19,NaN,138291365,16
76,Sing,5-May,66380,0.73,NaN,270395425,20
218,Hidden Figures,3-Nov,220283,NaN,NaN,169607287,46
664,Lowriders,23-Jun,123605,0.01,NaN,6303560,7


In [667]:
revenue5=revenue5[revenue5['Avg'].notnull()]
revenue5=revenue5.reset_index(drop=True)

In [666]:
revenue5[revenue5['Movie_Name']=='Lowriders']

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number
658,Lowriders,12-May,3028385,NaN,10266.0,3028385,1
659,Lowriders,19-May,1559750,-0.49,4273.0,4588135,2
660,Lowriders,26-May,964870,-0.38,2889.0,5553005,3
661,Lowriders,2-Jun,309415,-0.68,1729.0,5862420,4
662,Lowriders,9-Jun,194630,-0.37,1819.0,6057050,5
663,Lowriders,16-Jun,122905,-0.37,3073.0,6179955,6
664,Lowriders,23-Jun,123605,0.01,NaN,6303560,7


In [668]:
revenue5_final_5=revenue5.fillna(0)

In [669]:
revenue5_final_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2476 entries, 0 to 2475
Data columns (total 7 columns):
Movie_Name           2476 non-null object
Date                 2476 non-null object
Weekly_Gross         2476 non-null int64
Percentage_Change    2476 non-null float64
Avg                  2476 non-null float64
Gross_to_Date        2476 non-null int64
Week_Number          2476 non-null int64
dtypes: float64(2), int64(3), object(2)
memory usage: 135.5+ KB


## import data year

In [670]:
revenue5_final_5.head(3)

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number
0,Guardians of the Galaxy,1-Aug,134390839,0.00,32939.0,134390839,1
1,Guardians of the Galaxy,8-Aug,63154982,-0.53,15449.0,197545821,2
2,Guardians of the Galaxy,15-Aug,36708036,-0.42,9929.0,234253857,3


In [671]:
revenue5_final_5['Year']=0

In [672]:
revenue5_final_5.head(3)

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year
0,Guardians of the Galaxy,1-Aug,134390839,0.00,32939.0,134390839,1,0
1,Guardians of the Galaxy,8-Aug,63154982,-0.53,15449.0,197545821,2,0
2,Guardians of the Galaxy,15-Aug,36708036,-0.42,9929.0,234253857,3,0


In [678]:
#Append data fron another dataset (Kaggle Dataset)
for i in range(0,len(movie)):
    topic=movie['Title'][i]
    revenue5_final_5[revenue5_final_5['Movie_Name']==topic]
    revenue5_final_5['Year'][revenue5_final_5['Movie_Name']==topic]=movie['Year'][movie['Title']==topic].iloc[0]

C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [679]:
revenue5_final_5.head()

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year
0,Guardians of the Galaxy,1-Aug,134390839,0.00,32939.0,134390839,1,2014
1,Guardians of the Galaxy,8-Aug,63154982,-0.53,15449.0,197545821,2,2014
2,Guardians of the Galaxy,15-Aug,36708036,-0.42,9929.0,234253857,3,2014
3,Guardians of the Galaxy,22-Aug,24043411,-0.35,7132.0,258297268,4,2014
4,Guardians of the Galaxy,29-Aug,26110586,0.09,7542.0,284407854,5,2014


In [680]:
#check who got the wrong 'Year' values
revenue5_final_5['Movie_Name'][[len(str(revenue5_final_5['Year'][i]))!=4 for i in range(0,len(revenue5_final_5))]].unique()

array([], dtype=object)

In [682]:
revenue5_final_5[revenue5_final_5['Week_Number']==0]

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year
55,Sing,16-Dec,20612640,0.0,5125.0,20612640,0,2016
138,Passengers,16-Dec,7325201,0.0,2106.0,7325201,0,2016
236,Moana,18-Nov,25448873,0.0,6567.0,25448873,0,2016
454,Assassin's Creed,16-Dec,7492398,0.0,2582.0,7492398,0,2016
545,Interstellar,31-Oct,2151453,0.0,8640.0,2151453,0,2014
640,The Founder,13-Jan,1266,0.0,1266.0,1266,0,2016
925,Patriots Day,16-Dec,95412,0.0,13630.0,95412,0,2016
976,Allied,18-Nov,5024304,0.0,1590.0,5024304,0,2016
1021,Pirates of the Caribbean: At World's End,18-May,13240044,0.0,4271.0,13240044,0,2007
1063,Inglourious Basterds,14-Aug,15761,0.0,563.0,15761,0,2009


In [683]:
revenue5_final_5_v3=revenue5_final_5
df=revenue5_final_5

topics=df['Movie_Name'].unique()
for topic in topics:
    min_week=min(df['Week_Number'][df['Movie_Name']==topic])
    if min_week<1:
        df['Week_Number'][df['Movie_Name']==topic]=(df['Week_Number'][df['Movie_Name']==topic]+1)
        
        
revenue5_final_5=df

C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Check missing value

In [687]:
revenue5_final_5[revenue5_final_5['Year']==0]

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year


In [688]:
pd.unique(revenue5_final_5['Movie_Name'][revenue5_final_5['Year']==0])

array([], dtype=object)

In [689]:
#revenue5_final_5['Year'][revenue5_final_5['Movie_Name']=='Detour']=2016

## Input time data

### Changing Time format

In [690]:
revenue5_final_5['Date'][revenue5_final_5['Date']=="Feb-29"]="29-Feb"

C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Check missing value --- Date

In [691]:
revenue5_final_5[[len(revenue5_final_5['Date'][i])<5 for i in range(0,len(revenue5_final_5))]]

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year


### input since_str

#### Input first week

In [695]:
# insert since_str
revenue5_final_5['since_str']=''
revenue5_final_5['since_str'][revenue5_final_5['Week_Number']==1]=np.array([(str(revenue5_final_5['Year'][revenue5_final_5['Week_Number']==1][i])+
                                 '-'+str(switch_mon(revenue5_final_5['Date'][revenue5_final_5['Week_Number']==1][i].split("-")[1]))+
                                 '-'+revenue5_final_5['Date'][revenue5_final_5['Week_Number']==1][i].split("-")[0]) 
                                        for i in (revenue5_final_5['since_str'][revenue5_final_5['Week_Number']==1].index)])


C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [696]:
revenue5_final_5.head()

,Movie_Name,Date,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str
0,Guardians of the Galaxy,1-Aug,134390839,0.00,32939.0,134390839,1,2014,2014-08-1
1,Guardians of the Galaxy,8-Aug,63154982,-0.53,15449.0,197545821,2,2014,
2,Guardians of the Galaxy,15-Aug,36708036,-0.42,9929.0,234253857,3,2014,
3,Guardians of the Galaxy,22-Aug,24043411,-0.35,7132.0,258297268,4,2014,
4,Guardians of the Galaxy,29-Aug,26110586,0.09,7542.0,284407854,5,2014,
5,Guardians of the Galaxy,5-Sep,13476978,-0.48,4184.0,297884832,6,2014,
6,Guardians of the Galaxy,12-Sep,10604199,-0.21,3416.0,308489031,7,2014,
7,Guardians of the Galaxy,19-Sep,6914244,-0.35,2429.0,315403275,8,2014,
8,Guardians of the Galaxy,26-Sep,4923129,-0.29,2009.0,320326404,9,2014,
9,Guardians of the Galaxy,3-Oct,3943500,-0.20,2082.0,324269904,10,2014,


In [697]:
# insert since_date
revenue5_final_5['since_date']=''
revenue5_final_5['since_date'][revenue5_final_5['Week_Number']==1]=np.array(
    [datetime.datetime.strptime(revenue5_final_5['since_str'][revenue5_final_5['Week_Number']==1][i],"%Y-%m-%d") 
     for i in (revenue5_final_5['since_str'][revenue5_final_5['Week_Number']==1].index)])


C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [698]:
revenue5_final_5=revenue5_final_5.drop(['Date'],axis=1)
#Year	since_str	since_date

In [699]:
revenue5_final_5_v2=revenue5_final_5

In [700]:
#insert rows of two month before movie release
df=revenue5_final_5
topics=df['Movie_Name'].unique()
for topic in topics:
    min_week=min(df['Week_Number'][df['Movie_Name']==topic])
    for i in range(int(min_week-8),int(min_week)):
        df.loc[-1] = [topic, 0.0, 0.0,0.0,0.0,float(i),'2012','','']
        df.index = df.index + 1  # shifting index
        df = df.sort_index()  # sorting by index

df=df.sort_values(by=['Movie_Name','Week_Number'])
df=df.reset_index(drop=True)
revenue5_final_5=df

In [702]:
revenue5_final_5

,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date
0,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-7.0,2012,,
1,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-6.0,2012,,
2,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-5.0,2012,,
3,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-4.0,2012,,
4,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-3.0,2012,,
5,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-2.0,2012,,
6,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-1.0,2012,,
7,10 Cloverfield Lane,0.0,0.00,0.0,0.0,0.0,2012,,
8,10 Cloverfield Lane,32676639.0,0.00,9636.0,32676639.0,1.0,2016,2016-03-11,2016-03-11 00:00:00
9,10 Cloverfield Lane,17334158.0,-0.47,5058.0,50010797.0,2.0,2016,,


In [703]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3700 entries, 0 to 3699
Data columns (total 9 columns):
Movie_Name           3700 non-null object
Weekly_Gross         3700 non-null float64
Percentage_Change    3700 non-null float64
Avg                  3700 non-null float64
Gross_to_Date        3700 non-null float64
Week_Number          3700 non-null float64
Year                 3700 non-null object
since_str            3700 non-null object
since_date           3700 non-null object
dtypes: float64(5), object(4)
memory usage: 260.2+ KB


In [634]:
revenue5_final_5[revenue5_final_5['Movie_Name']=='Maudie']

,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date
2252,Maudie,0.0,0.00,0.0,0.0,-4.0,2012,,
2253,Maudie,0.0,0.00,0.0,0.0,-3.0,2012,,
2254,Maudie,0.0,0.00,0.0,0.0,-2.0,2012,,
2255,Maudie,0.0,0.00,0.0,0.0,-1.0,2012,,
2256,Maudie,0.0,0.00,0.0,0.0,0.0,2012,,
2257,Maudie,0.0,0.00,0.0,0.0,1.0,2012,,
2258,Maudie,0.0,0.00,0.0,0.0,2.0,2012,,
2259,Maudie,0.0,0.00,0.0,0.0,3.0,2012,,
2260,Maudie,111911.0,0.00,4663.0,2699007.0,4.0,2016,,
2261,Maudie,133253.0,0.19,4759.0,2832260.0,6.0,2016,,


In [643]:
#remove wrong data
revenue5_final_5=revenue5_final_5[revenue5_final_5['Movie_Name']!='Maudie']
revenue5_final_5=revenue5_final_5.reset_index(drop=True)

In [704]:
for topic in pd.unique(revenue5_final_5['Movie_Name']):
    for i in revenue5_final_5['Week_Number'][(revenue5_final_5['Movie_Name']==topic)]:
        revenue5_final_5['since_date'][(revenue5_final_5['Movie_Name']==topic) & (revenue5_final_5['Week_Number']==i)]=(
            revenue5_final_5['since_date'][(revenue5_final_5['Movie_Name']==topic) & (revenue5_final_5['Week_Number']==1)].iloc[0]+
            (i-1)*datetime.timedelta(7))

C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [707]:
revenue5_final_5

,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date
0,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-7.0,2012,,2016-01-15 00:00:00
1,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-6.0,2012,,2016-01-22 00:00:00
2,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-5.0,2012,,2016-01-29 00:00:00
3,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-4.0,2012,,2016-02-05 00:00:00
4,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-3.0,2012,,2016-02-12 00:00:00
5,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-2.0,2012,,2016-02-19 00:00:00
6,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-1.0,2012,,2016-02-26 00:00:00
7,10 Cloverfield Lane,0.0,0.00,0.0,0.0,0.0,2012,,2016-03-04 00:00:00
8,10 Cloverfield Lane,32676639.0,0.00,9636.0,32676639.0,1.0,2016,2016-03-11,2016-03-11 00:00:00
9,10 Cloverfield Lane,17334158.0,-0.47,5058.0,50010797.0,2.0,2016,,2016-03-18 00:00:00


In [708]:
revenue5_final_5['since_str']=np.array([str(revenue5_final_5['since_date'][i]).split(' ')[0] for i in range(0,len(revenue5_final_5))])

In [709]:
revenue5_final_5['since_str']=np.array([str(revenue5_final_5['since_date'][i]).split(' ')[0] 
                                        for i in range(0,len(revenue5_final_5))])

revenue5_final_5['until_date']=revenue5_final_5['since_date']+datetime.timedelta(7)

revenue5_final_5['until_str']=np.array([str(revenue5_final_5['until_date'][i]).split(' ')[0] 
                                        for i in range(0,len(revenue5_final_5))])

revenue5_final_5['Month']=np.array([(revenue5_final_5['since_str'][i]).split('-')[1] 
                                        for i in range(0,len(revenue5_final_5))])

revenue5_final_5['Day']=np.array([(revenue5_final_5['since_str'][i]).split('-')[2] 
                                        for i in range(0,len(revenue5_final_5))])

revenue5_final_5['Year']=np.array([(revenue5_final_5['since_str'][i]).split('-')[0] 
                                        for i in range(0,len(revenue5_final_5))])

In [710]:
revenue5_final_5

,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date,until_date,until_str,Month,Day
0,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-7.0,2016,2016-01-15,2016-01-15 00:00:00,2016-01-22,2016-01-22,01,15
1,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-6.0,2016,2016-01-22,2016-01-22 00:00:00,2016-01-29,2016-01-29,01,22
2,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-5.0,2016,2016-01-29,2016-01-29 00:00:00,2016-02-05,2016-02-05,01,29
3,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-4.0,2016,2016-02-05,2016-02-05 00:00:00,2016-02-12,2016-02-12,02,05
4,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-3.0,2016,2016-02-12,2016-02-12 00:00:00,2016-02-19,2016-02-19,02,12
5,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-2.0,2016,2016-02-19,2016-02-19 00:00:00,2016-02-26,2016-02-26,02,19
6,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-1.0,2016,2016-02-26,2016-02-26 00:00:00,2016-03-04,2016-03-04,02,26
7,10 Cloverfield Lane,0.0,0.00,0.0,0.0,0.0,2016,2016-03-04,2016-03-04 00:00:00,2016-03-11,2016-03-11,03,04
8,10 Cloverfield Lane,32676639.0,0.00,9636.0,32676639.0,1.0,2016,2016-03-11,2016-03-11 00:00:00,2016-03-18,2016-03-18,03,11
9,10 Cloverfield Lane,17334158.0,-0.47,5058.0,50010797.0,2.0,2016,2016-03-18,2016-03-18 00:00:00,2016-03-25,2016-03-25,03,18


In [711]:
revenue5_final_5.to_csv("MovieRevenue_Cleaned_1.csv",index=False)

# ---------------Merging and Slice data for Twitter Web Crawler----------------

In [712]:
revenue1_c=pd.read_csv('MovieRevenue_Cleaned_1.csv',encoding = "ISO-8859-1")
revenue2_c=pd.read_csv('MovieRevenue_Cleaned_2.csv',encoding = "ISO-8859-1")
revenue3_c=pd.read_csv('MovieRevenue_Cleaned_3.csv',encoding = "ISO-8859-1")
revenue4_c=pd.read_csv('MovieRevenue_Cleaned_4.csv',encoding = "ISO-8859-1")
revenue5_c=pd.read_csv('MovieRevenue_Cleaned_5.csv',encoding = "ISO-8859-1")

In [720]:
revenue4_c

,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date,until_date,until_str,Month,Day
0,10 Years,0.0,0.00,0.0,0.0,-7.0,2011,2011-07-20,2011-07-20 00:00:00,2011-07-27,2011-07-27,7,20
1,10 Years,0.0,0.00,0.0,0.0,-6.0,2011,2011-07-27,2011-07-27 00:00:00,2011-08-03,2011-08-03,7,27
2,10 Years,0.0,0.00,0.0,0.0,-5.0,2011,2011-08-03,2011-08-03 00:00:00,2011-08-10,2011-08-10,8,3
3,10 Years,0.0,0.00,0.0,0.0,-4.0,2011,2011-08-10,2011-08-10 00:00:00,2011-08-17,2011-08-17,8,10
4,10 Years,0.0,0.00,0.0,0.0,-3.0,2011,2011-08-17,2011-08-17 00:00:00,2011-08-24,2011-08-24,8,17
5,10 Years,0.0,0.00,0.0,0.0,-2.0,2011,2011-08-24,2011-08-24 00:00:00,2011-08-31,2011-08-31,8,24
6,10 Years,0.0,0.00,0.0,0.0,-1.0,2011,2011-08-31,2011-08-31 00:00:00,2011-09-07,2011-09-07,8,31
7,10 Years,0.0,0.00,0.0,0.0,0.0,2011,2011-09-07,2011-09-07 00:00:00,2011-09-14,2011-09-14,9,7
8,10 Years,30915.0,0.00,10305.0,30915.0,1.0,2011,2011-09-14,2011-09-14 00:00:00,2011-09-21,2011-09-21,9,14
9,10 Years,107149.0,2.47,2492.0,138064.0,2.0,2011,2011-09-21,2011-09-21 00:00:00,2011-09-28,2011-09-28,9,21


In [715]:
revenue1_c.append(revenue2_c).append(revenue3_c).append(revenue4_c).append(revenue5_c)

,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date,until_date,until_str,Month,Day
0,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-7.0,2016,2016-01-15,2016-01-15 00:00:00,2016-01-22,2016-01-22,1,15
1,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-6.0,2016,2016-01-22,2016-01-22 00:00:00,2016-01-29,2016-01-29,1,22
2,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-5.0,2016,2016-01-29,2016-01-29 00:00:00,2016-02-05,2016-02-05,1,29
3,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-4.0,2016,2016-02-05,2016-02-05 00:00:00,2016-02-12,2016-02-12,2,5
4,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-3.0,2016,2016-02-12,2016-02-12 00:00:00,2016-02-19,2016-02-19,2,12
5,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-2.0,2016,2016-02-19,2016-02-19 00:00:00,2016-02-26,2016-02-26,2,19
6,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-1.0,2016,2016-02-26,2016-02-26 00:00:00,2016-03-04,2016-03-04,2,26
7,10 Cloverfield Lane,0.0,0.00,0.0,0.0,0.0,2016,2016-03-04,2016-03-04 00:00:00,2016-03-11,2016-03-11,3,4
8,10 Cloverfield Lane,32676639.0,0.00,9636.0,32676639.0,1.0,2016,2016-03-11,2016-03-11 00:00:00,2016-03-18,2016-03-18,3,11
9,10 Cloverfield Lane,17334158.0,-0.47,5058.0,50010797.0,2.0,2016,2016-03-18,2016-03-18 00:00:00,2016-03-25,2016-03-25,3,18


In [716]:
data=revenue1_c.append(revenue2_c).append(revenue3_c).append(revenue4_c).append(revenue5_c)

In [722]:
data=data.reset_index()

In [723]:
data

,index,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date,until_date,until_str,Month,Day
0,0,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-7.0,2016,2016-01-15,2016-01-15 00:00:00,2016-01-22,2016-01-22,1,15
1,1,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-6.0,2016,2016-01-22,2016-01-22 00:00:00,2016-01-29,2016-01-29,1,22
2,2,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-5.0,2016,2016-01-29,2016-01-29 00:00:00,2016-02-05,2016-02-05,1,29
3,3,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-4.0,2016,2016-02-05,2016-02-05 00:00:00,2016-02-12,2016-02-12,2,5
4,4,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-3.0,2016,2016-02-12,2016-02-12 00:00:00,2016-02-19,2016-02-19,2,12
5,5,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-2.0,2016,2016-02-19,2016-02-19 00:00:00,2016-02-26,2016-02-26,2,19
6,6,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-1.0,2016,2016-02-26,2016-02-26 00:00:00,2016-03-04,2016-03-04,2,26
7,7,10 Cloverfield Lane,0.0,0.00,0.0,0.0,0.0,2016,2016-03-04,2016-03-04 00:00:00,2016-03-11,2016-03-11,3,4
8,8,10 Cloverfield Lane,32676639.0,0.00,9636.0,32676639.0,1.0,2016,2016-03-11,2016-03-11 00:00:00,2016-03-18,2016-03-18,3,11
9,9,10 Cloverfield Lane,17334158.0,-0.47,5058.0,50010797.0,2.0,2016,2016-03-18,2016-03-18 00:00:00,2016-03-25,2016-03-25,3,18


In [724]:
#data.to_csv("MovieRevenue_All.csv",index=False)

In [725]:
data['Genre']=''
data['Director']=''
data['Actors']=''
data['Rating']=0
data['Revenue']=0.0
data['Runtime']=''
data['Voting']=''
data['Metascore']=0



In [826]:
data

,index,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date,...,Month,Day,Genre,Director,Actors,Rating,Revenue,Runtime,Voting,Metascore
0,0,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-7.0,2016,2016-01-15,2016-01-15 00:00:00,...,1,15,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0
1,1,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-6.0,2016,2016-01-22,2016-01-22 00:00:00,...,1,22,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0
2,2,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-5.0,2016,2016-01-29,2016-01-29 00:00:00,...,1,29,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0
3,3,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-4.0,2016,2016-02-05,2016-02-05 00:00:00,...,2,5,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0
4,4,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-3.0,2016,2016-02-12,2016-02-12 00:00:00,...,2,12,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0
5,5,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-2.0,2016,2016-02-19,2016-02-19 00:00:00,...,2,19,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0
6,6,10 Cloverfield Lane,0.0,0.00,0.0,0.0,-1.0,2016,2016-02-26,2016-02-26 00:00:00,...,2,26,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0
7,7,10 Cloverfield Lane,0.0,0.00,0.0,0.0,0.0,2016,2016-03-04,2016-03-04 00:00:00,...,3,4,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0
8,8,10 Cloverfield Lane,32676639.0,0.00,9636.0,32676639.0,1.0,2016,2016-03-11,2016-03-11 00:00:00,...,3,11,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0
9,9,10 Cloverfield Lane,17334158.0,-0.47,5058.0,50010797.0,2.0,2016,2016-03-18,2016-03-18 00:00:00,...,3,18,"Drama,Horror,Mystery",Dan Trachtenberg,"John Goodman, Mary Elizabeth Winstead, John Ga...",7.2,71.9,104,192968,76.0


In [728]:
data2=data
revenue5_final_5=data

for i in range(0,len(movie)):
    topic=movie['Title'][i]
    #revenue5_final_5[revenue5_final_5['Movie_Name']==topic]
    revenue5_final_5['Genre'][revenue5_final_5['Movie_Name']==topic]=movie['Genre'][movie['Title']==topic].iloc[0]
    revenue5_final_5['Director'][revenue5_final_5['Movie_Name']==topic]=movie['Director'][movie['Title']==topic].iloc[0]
    revenue5_final_5['Actors'][revenue5_final_5['Movie_Name']==topic]=movie['Actors'][movie['Title']==topic].iloc[0]
    revenue5_final_5['Rating'][revenue5_final_5['Movie_Name']==topic]=movie['Rating'][movie['Title']==topic].iloc[0]
    revenue5_final_5['Revenue'][revenue5_final_5['Movie_Name']==topic]=movie['Revenue (Millions)'][movie['Title']==topic].iloc[0]
    revenue5_final_5['Runtime'][revenue5_final_5['Movie_Name']==topic]=movie['Runtime (Minutes)'][movie['Title']==topic].iloc[0]
    revenue5_final_5['Voting'][revenue5_final_5['Movie_Name']==topic]=movie['Votes'][movie['Title']==topic].iloc[0]
    revenue5_final_5['Metascore'][revenue5_final_5['Movie_Name']==topic]=movie['Metascore'][movie['Title']==topic].iloc[0]
        
        
data=revenue5_final_5

C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Chih-Hsiang\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyW

In [730]:
dataN=data

In [750]:
dataN['Movie_Name'][data['Genre']==''].unique()

array(['Detour'], dtype=object)

In [752]:
dataFinal['Movie_Name'][dataFinal['Director']==''].unique()

array([], dtype=object)

In [751]:
dataFinal=dataN[dataN['Movie_Name']!='Detour']
dataFinal=dataFinal.reset_index(drop=True)

In [786]:
dataFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19046 entries, 0 to 19045
Data columns (total 22 columns):
index                19046 non-null int64
Movie_Name           19046 non-null object
Weekly_Gross         19046 non-null float64
Percentage_Change    19046 non-null float64
Avg                  19046 non-null float64
Gross_to_Date        19046 non-null float64
Week_Number          19046 non-null float64
Year                 19046 non-null int64
since_str            19046 non-null object
since_date           19046 non-null object
until_date           19046 non-null object
until_str            19046 non-null object
Month                19046 non-null int64
Day                  19046 non-null int64
Genre                19046 non-null object
Director             19046 non-null object
Actors               19046 non-null object
Rating               19030 non-null float64
Revenue              19046 non-null float64
Runtime              19030 non-null float64
Voting               19030 

In [783]:
np.median(dataFinalFinal['Revenue'][dataFinalFinal['Revenue'].notnull()])

59.890000000000001

In [784]:
dataFinalFinal['Revenue']=dataFinalFinal['Revenue'].fillna(np.median(dataFinalFinal['Revenue'][dataFinalFinal['Revenue'].notnull()]))

In [785]:
dataFinalFinal['Metascore']=dataFinalFinal['Metascore'].fillna(np.median(dataFinalFinal['Metascore'][dataFinalFinal['Metascore'].notnull()]))

In [797]:
dataFinalFinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19046 entries, 0 to 19045
Data columns (total 22 columns):
index                19046 non-null int64
Movie_Name           19046 non-null object
Weekly_Gross         19046 non-null float64
Percentage_Change    19046 non-null float64
Avg                  19046 non-null float64
Gross_to_Date        19046 non-null float64
Week_Number          19046 non-null float64
Year                 19046 non-null int64
since_str            19046 non-null object
since_date           19046 non-null object
until_date           19046 non-null object
until_str            19046 non-null object
Month                19046 non-null int64
Day                  19046 non-null int64
Genre                19046 non-null object
Director             19046 non-null object
Actors               19046 non-null object
Rating               19046 non-null float64
Revenue              19046 non-null float64
Runtime              19046 non-null object
Voting               19046 n

In [795]:
dataFinalFinal[dataFinalFinal['Rating'].isnull()]

,index,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date,...,Month,Day,Genre,Director,Actors,Rating,Revenue,Runtime,Voting,Metascore
16906,3061,Riddick,0.0,0.00,0.0,0.0,-7.0,2013,2013-07-12,2013-07-12 00:00:00,...,7,12,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0
16907,3062,Riddick,0.0,0.00,0.0,0.0,-6.0,2013,2013-07-19,2013-07-19 00:00:00,...,7,19,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0
16908,3063,Riddick,0.0,0.00,0.0,0.0,-5.0,2013,2013-07-26,2013-07-26 00:00:00,...,7,26,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0
16909,3064,Riddick,0.0,0.00,0.0,0.0,-4.0,2013,2013-08-02,2013-08-02 00:00:00,...,8,2,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0
16910,3065,Riddick,0.0,0.00,0.0,0.0,-3.0,2013,2013-08-09,2013-08-09 00:00:00,...,8,9,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0
16911,3066,Riddick,0.0,0.00,0.0,0.0,-2.0,2013,2013-08-16,2013-08-16 00:00:00,...,8,16,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0
16912,3067,Riddick,0.0,0.00,0.0,0.0,-1.0,2013,2013-08-23,2013-08-23 00:00:00,...,8,23,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0
16913,3068,Riddick,0.0,0.00,0.0,0.0,0.0,2013,2013-08-30,2013-08-30 00:00:00,...,8,30,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0
16914,3069,Riddick,24266375.0,0.00,7810.0,24266375.0,1.0,2013,2013-09-06,2013-09-06 00:00:00,...,9,6,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0
16915,3070,Riddick,9241915.0,-0.62,2965.0,33508290.0,2.0,2013,2013-09-13,2013-09-13 00:00:00,...,9,13,"Action,Sci-Fi,Thriller",David Twohy,"Vin Diesel, Karl Urban, Katee Sackhoff, Jordi ...",NaN,59.89,119,132098,2009.0


In [779]:
dataFinalFinal['Voting']=pd.to_numeric(dataFinalFinal['Voting'], errors='coerce')

In [768]:
dataFinal.to_csv("MovieRevenue_All_v2.csv",index=False)

In [798]:
dataFinalFinal.to_csv("MovieRevenue_All_v3.csv",index=False)

In [825]:
dataFinalFinal1=dataFinalFinal[dataFinalFinal['Year']>=2016]
dataFinalFinal1=dataFinalFinal1.reset_index(drop=True)
dataFinalFinal1.to_csv("MovieRevenue_All_slice0.csv",index=False)

In [812]:
dataFinalFinal1

,index,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date,...,Month,Day,Genre,Director,Actors,Rating,Revenue,Runtime,Voting,Metascore
0,75,300,0.0,0.00,0.0,0.0,-7.0,2006,2006-01-12,2006-01-12 00:00:00,...,1,12,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0
1,76,300,0.0,0.00,0.0,0.0,-6.0,2006,2006-01-19,2006-01-19 00:00:00,...,1,19,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0
2,77,300,0.0,0.00,0.0,0.0,-5.0,2006,2006-01-26,2006-01-26 00:00:00,...,1,26,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0
3,78,300,0.0,0.00,0.0,0.0,-4.0,2006,2006-02-02,2006-02-02 00:00:00,...,2,2,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0
4,79,300,0.0,0.00,0.0,0.0,-3.0,2006,2006-02-09,2006-02-09 00:00:00,...,2,9,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0
5,80,300,0.0,0.00,0.0,0.0,-2.0,2006,2006-02-16,2006-02-16 00:00:00,...,2,16,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0
6,81,300,0.0,0.00,0.0,0.0,-1.0,2006,2006-02-23,2006-02-23 00:00:00,...,2,23,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0
7,82,300,0.0,0.00,0.0,0.0,0.0,2006,2006-03-02,2006-03-02 00:00:00,...,3,2,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0
8,83,300,96288328.0,0.00,31031.0,96288328.0,1.0,2006,2006-03-09,2006-03-09 00:00:00,...,3,9,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0
9,84,300,45555327.0,-0.53,13931.0,141843655.0,2.0,2006,2006-03-16,2006-03-16 00:00:00,...,3,16,"Action,Fantasy,War",Zack Snyder,"Gerard Butler, Lena Headey, David Wenham, Domi...",7.7,210.59,117,637104,52.0


In [803]:
dataFinalFinal1.to_csv("MovieRevenue_All_slice1.csv",index=False)

,index,Movie_Name,Weekly_Gross,Percentage_Change,Avg,Gross_to_Date,Week_Number,Year,since_str,since_date,...,Month,Day,Genre,Director,Actors,Rating,Revenue,Runtime,Voting,Metascore
111,111,A Monster Calls,2732136.0,67.40,1794.0,2823628.0,3.0,2017,2017-01-06,2017-01-06 00:00:00,...,1,6,"Drama,Fantasy",J.A. Bayona,"Lewis MacDougall, Sigourney Weaver, Felicity J...",7.5,3.73,108,39134,76.0
112,112,A Monster Calls,887171.0,-0.68,586.0,3710799.0,4.0,2017,2017-01-13,2017-01-13 00:00:00,...,1,13,"Drama,Fantasy",J.A. Bayona,"Lewis MacDougall, Sigourney Weaver, Felicity J...",7.5,3.73,108,39134,76.0
113,113,A Monster Calls,30024.0,-0.97,715.0,3740823.0,5.0,2017,2017-01-20,2017-01-20 00:00:00,...,1,20,"Drama,Fantasy",J.A. Bayona,"Lewis MacDougall, Sigourney Weaver, Felicity J...",7.5,3.73,108,39134,76.0
129,129,Allied,134059.0,-0.52,1108.0,39854249.0,8.0,2017,2017-01-06,2017-01-06 00:00:00,...,1,6,"Action,Drama,Romance",Robert Zemeckis,"Brad Pitt, Marion Cotillard, Jared Harris, Vin...",7.1,40.07,124,78079,60.0
130,130,Allied,161201.0,0.20,926.0,40015450.0,9.0,2017,2017-01-13,2017-01-13 00:00:00,...,1,13,"Action,Drama,Romance",Robert Zemeckis,"Brad Pitt, Marion Cotillard, Jared Harris, Vin...",7.1,40.07,124,78079,60.0
131,131,Allied,82614.0,-0.49,688.0,40098064.0,10.0,2017,2017-01-20,2017-01-20 00:00:00,...,1,20,"Action,Drama,Romance",Robert Zemeckis,"Brad Pitt, Marion Cotillard, Jared Harris, Vin...",7.1,40.07,124,78079,60.0
148,148,Arrival,1258058.0,-0.48,3302.0,94520580.0,9.0,2017,2017-01-06,2017-01-06 00:00:00,...,1,6,"Drama,Mystery,Sci-Fi",Denis Villeneuve,"Amy Adams, Jeremy Renner, Forest Whitaker,Mich...",8.0,100.50,116,340798,81.0
149,149,Arrival,829052.0,-0.34,3356.0,95349632.0,10.0,2017,2017-01-13,2017-01-13 00:00:00,...,1,13,"Drama,Mystery,Sci-Fi",Denis Villeneuve,"Amy Adams, Jeremy Renner, Forest Whitaker,Mich...",8.0,100.50,116,340798,81.0
150,150,Arrival,505567.0,-0.39,2809.0,95855199.0,11.0,2017,2017-01-20,2017-01-20 00:00:00,...,1,20,"Drama,Mystery,Sci-Fi",Denis Villeneuve,"Amy Adams, Jeremy Renner, Forest Whitaker,Mich...",8.0,100.50,116,340798,81.0
151,151,Arrival,1986436.0,2.93,1627.0,97841635.0,12.0,2017,2017-01-27,2017-01-27 00:00:00,...,1,27,"Drama,Mystery,Sci-Fi",Denis Villeneuve,"Amy Adams, Jeremy Renner, Forest Whitaker,Mich...",8.0,100.50,116,340798,81.0
